In [1]:
import numpy as np
import cv2
import sys
import os

In [2]:
name = "AB_HetY1Hpilot3-16_5mm_3AT_X-gal_7d.JPG.JPG"
for root, dirs, files in os.walk("data/"):
    if name in files:
        print(os.path.join(root, name))

In [3]:
def resize_im(im,scale):
    #percent by which the image is resized
    scale_percent = 20

    #calculate the 50 percent of original dimensions
    width = int(im.shape[1] * scale / 100)
    height = int(im.shape[0] * scale / 100)

    # dsize
    dsize = (width, height)

    # resize image
    test_output = cv2.resize(im, dsize)
    return test_output

In [4]:
def detect_circles(im):
    output = im#resize_im(im, 50)
    gray = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5)
    rows = gray.shape[0]
    """ 
    If a pixel gradient is higher than the upper threshold(param1), the pixel is accepted as an edge
    If a pixel gradient value is below the lower threshold(param2), then it is rejected.
    If the pixel gradient is between the two thresholds, then it will be accepted only if it is connected to a pixel that is above the upper threshold.
    Canny recommended a upper:lower ratio between 2:1 and 3:1.
    """
    circles = cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT,
                               1,#dp inverse, usually one
                               75,#min distance between circles
                               param1=15,#50,
                               param2=12,
                               minRadius=0,
                               maxRadius=45)

    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:
        # draw the outer circle
        #cv2.circle(output,(i[0],i[1]),i[2],(0,255,0),2)
        # draw the center of the circle
        cv2.circle(output,(i[0],i[1]),2,(0,0,255),3)

        #print("circles : {}".format(circles))
    """if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            center = (i[0], i[1])
            # circle center
            cv2.circle(output, center, 1, (0, 100, 100), 3)
            # circle outline
            radius = i[2]
            cv2.circle(output, center, radius, (255, 0, 255), 3)"""
    return output,circles

In [5]:
# import personal image
image = cv2.imread("data/AB_HetY1Hpilot3-9_5mm_3AT_X-gal_7d.JPG")

In [6]:
def getHistogramProjection(img):
    histogram = img.copy()
    histogram[histogram == 255] = 1
    return np.sum(histogram,axis=0),np.sum(histogram,axis=1)

In [7]:
img = cv2.Canny(image,20,30)
template1 = cv2.imread('./mahir/Heterodimer-Y1H-Analysis-main/upperLeft.png',0)
template2 = cv2.imread('./mahir/Heterodimer-Y1H-Analysis-main/bottomRight.png',0)

res = cv2.matchTemplate(img,template1,cv2.TM_CCORR_NORMED)
res1= cv2.matchTemplate(img,template2,cv2.TM_CCORR_NORMED)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
top_left = np.add(max_loc,[140,40])
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res1)
bottom_right = np.add(max_loc,[0,140])
img = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
# cv2.imshow('first crop.png',img)
# cv2.waitKey()
# cv2.rectangle(img,tuple(top_left), tuple(bottom_right), 255, 2)
# cv2.imwrite('crop.jpg',img1[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]])
# cv2.waitKey(0)
# print(top_left,bottom_right)

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# hist,bins = np.histogram(img.ravel(),256,[0,256])
gray = ((gray < 110) * 255).astype(np.uint8)
# cv2.imshow('gray',gray)
# cv2.waitKey()
xProj,yProj=getHistogramProjection(gray)
xNonZero = [i for i, elem in enumerate(xProj) if elem >30]
yNonZero= [i for i, elem in enumerate(yProj) if elem >30]
crop = img[yNonZero[0]:yNonZero[-1],xNonZero[0]:xNonZero[-1]]

In [9]:
cv2.namedWindow("Display window", cv2.WINDOW_AUTOSIZE)
cv2.imshow('passes ', resize_im(detect_circles(crop)[0],30))
cv2.waitKey(0)
#closing all open windows  
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [588]:
print(detect_circles(crop)[1].shape)

(1, 824, 3)


In [ ]:
#test2
im = image
imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gauss = cv2.GaussianBlur(imgray, (5, 5), 0)
ret, thresh = cv2.threshold(im_gauss, 127, 255, 0)
# get contours
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

contours_area = []
# calculate area and filter into new array
for con in contours:
    area = cv2.contourArea(con)
    if 1000 < area < 10000:
        contours_area.append(con)